In [2]:
from PIL import Image, ImageDraw, ImageFont
import os
import random
import string
import pandas as pd

In [3]:
BASE_PATH = '.'
DATA_PATH = '2018-12-03'
FONT_PATH = BASE_PATH + '/Fonts'
PATTERN_PATH = BASE_PATH + '/Patterns'
RESOURCE_PATH = BASE_PATH + "/Resources" + "/" + DATA_PATH
fonts = [f for f in os.listdir(FONT_PATH) if f.endswith('.ttf')]

image_w = 120
image_h = 100


colors = [(0, 0, 0), (255, 255, 255)]
complement = [(0, 0, 255) ,(255, 0, 0)]
patterns = ["1.png", "2.png", "3.png"]

In [5]:
case = 0
correct_answer = []
while case < 50000:
    case += 1
    forecolor = random.randrange(0, 2)
    backcolor = 1-forecolor
    img = Image.new('RGB', (image_w, image_h), color=colors[backcolor])
    #fnt_name = fonts[random.randrange(len(fonts))]
    fnt_name = "arialbi.ttf"
    rand_text = ''.join(random.choices(string.digits, k=4))
    # rand_text = ' '.join(random.choices(string.ascii_uppercase + string.digits, k=random.randrange(5, 9)))

    draw = ImageDraw.Draw(img)
    while True:
        try:
            #fnt_size= random.randrange(50, 80)
            fnt_size = 50
            fnt = ImageFont.truetype(FONT_PATH + '/' + fnt_name, fnt_size)
            text_w, text_h = draw.textsize(rand_text, font=fnt)
            text_x = (image_w-text_w)/2
            text_y = (image_h-text_h)/2
            draw.text((text_x, text_y), rand_text, font=fnt, fill=colors[forecolor])
        except ValueError:
            pass
        else:
            # print(case, fnt_name, fnt_size)
            break


    correct_answer.append(rand_text)
    title = str(case) + '.png'
    # title = rand_text.replace(" ","").lower() + "_" + fnt_name[:-5] + "_origin_" + str(case) + ".png"
    img.save(RESOURCE_PATH + "/origin/" + str(rand_text) + "_" + title, "png")

    img_convert = img.copy()
    pix_convert = img_convert.load()
    for level in range(3):
        for k in range(0, 5):
            center_x = random.randrange(0, image_w)
            center_y = random.randrange(0, image_h)
            radius = random.randrange(1, 100)*10
            for i in range(image_w):
                for j in range(image_h):
                    if (i-center_x)*(i-center_x) + (j-center_y)*(j-center_y) <= radius:
                            pix_convert[i, j] = (255-pix_convert[i, j][0], 255-pix_convert[i, j][1], 255-pix_convert[i, j][2])
        # title = rand_text.replace(" ","").lower() + "_" + fnt_name[:-5] + "_complement_lv+"+ str(level) + "_" + str(case) + ".png"
        img_convert.save(RESOURCE_PATH + "/complement/lv" + str(level) + "/" + str(rand_text) + "_lv" + str(level) + "_" + title, "png")

    for level in range(3):
        img_pattern = Image.open(PATTERN_PATH + '/' + patterns[level])
        pix_pattern = img_pattern.load()
        
        img_pattern_w, img_pattern_h = img_pattern.size
        start_x = random.randrange(0, img_pattern_w-image_w)
        start_y = random.randrange(0, img_pattern_h-image_h)
        
        img_erase = img.copy()
        img_add = img.copy()
        img_color = img.copy()
        pix_erase = img_erase.load()
        pix_add = img_add.load()
        pix_color = img_color.load()
        
        img_crop_pattern = Image.new("RGB", (image_w, image_h), color='white')
        pix_crop_pattern = img_crop_pattern.load()
        for i in range(image_w):
            for j in range(image_h):
                if pix_pattern[start_x + i, start_y + j][0] + pix_pattern[start_x + i, start_y + j][1] + pix_pattern[start_x + i, start_y + j][2] < 200:
                    pix_crop_pattern[i, j] = (0, 0, 0)
                    pix_erase[i, j] = colors[backcolor]
                    pix_add[i, j] = colors[forecolor]
                    pix_color[i, j] = complement[forecolor]
                    
        # title = rand_text.replace(" ","").lower() + "_" + fnt_name[:-5] + "_erase_lv+"+ str(level) + "_" + str(case) + ".png"
        img_erase.save(RESOURCE_PATH + "/erase/lv" + str(level) + "/" + str(rand_text) + "_lv" + str(level) + "_" + title, "png")
        # title = rand_text.replace(" ","").lower() + "_" + fnt_name[:-5] + "_add_lv+"+ str(level) + "_" + str(case) + ".png"
        img_add.save(RESOURCE_PATH + "/add/lv" + str(level) + "/" + str(rand_text) + "_lv" + str(level) + "_" + title, "png")
        # title = rand_text.replace(" ","").lower() + "_" + fnt_name[:-5] + "_add_color+"+ str(level) + "_" + str(case) + ".png"
        img_color.save(RESOURCE_PATH + "/add_color/lv" + str(level) + "/" + str(rand_text) + "_lv" + str(level) + "_" + title, "png")
        # title = rand_text.replace(" ","").lower() + "_" + fnt_name[:-5] + "_pattern_lv+"+ str(level) + "_" + str(case) + ".png"
        #img_crop_pattern.save(RESOURCE_PATH + "/pattern/lv" + str(level) + "/" + title, "png")
    
pd.Series(correct_answer).to_csv(RESOURCE_PATH + "/answer.csv" , header=None, index=None)
        